# Overview


In this notebook, we will make TSS annotation data that are used for the scATAC-seq peak annotation.


- First, we download gene annotation gff3 file from Ensembl database.
- Second, we convert gff3 file into bed file. During this process, the TSS information is extracted.


# !! Caution!!  

## 1) This is NOT part of CellOracle tutorial. 
- This notebook includes unusual usage of CellOracle. 
- The analysis might require expertise of python and DNA sequence analysis, but this notebook does not aim to explain them all, and please use this notebook by your responsibility.

## 2) This notebook was tested with Ensembl Guinea Pig data, but we do not guarantee the function works with other species or other database. 

- Please let us know using git hub issue if you have problem with this notebook.
- We can construct TSS annotation data and add them to CellOracle package. Please just let us know if you have a request for new reference genome.

# 0. Import libraries

In [1]:
import pandas as pd
import numpy as np

import os, sys
from tqdm.notebook import  tqdm

from pybedtools import BedTool
import genomepy

2022-12-27 16:40:15,172 [11281] WARNING  py.warnings:99: [JupyterRequire] /home/k/anaconda3/envs/pandas1/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)



In [2]:
import celloracle as co
from celloracle import motif_analysis as ma

co.__version__

'0.10.13'

# 1. Define custom functions to process gene annotation data.

Extract TSS information from gff3 file and get a bed file.

In [14]:
def parse_ens(x):
    dic = {}
    if ";" in str(x):
        for i in x.split(";"):
            key, val = i.split("=")
            dic[key] = val
    return dic

def get_tss_and_promoter_candidate_locus(data, n_downstream=500, n_upstream=500, clip_negative=True):
    data["TSS"] = data["start"]
    
    mRNA_in_reversed_strand = data.index[data["strand"] == "-"]
    data.loc[mRNA_in_reversed_strand, "TSS"] = \
        data.loc[mRNA_in_reversed_strand, "end"]
    
    data["promTSS_left"] = data["TSS"] - n_upstream
    data["promTSS_right"] = data["TSS"] + n_downstream
    
    data.loc[mRNA_in_reversed_strand, "promTSS_left"] = \
        data.loc[mRNA_in_reversed_strand, "TSS"] - n_downstream
    data.loc[mRNA_in_reversed_strand, "promTSS_right"] = \
        data.loc[mRNA_in_reversed_strand, "TSS"] + n_upstream
    
    if clip_negative:
        data.loc[data.index[data.promTSS_left < 0], "promTSS_left"] = 0
    
    return data

def merge_overlapping_peaks(df_):
    
    gene_symbol = df_.gene_symbol.unique()
    assert(len(gene_symbol) == 1)
    
    strand = df_.strand.unique()
    assert(len(strand) == 1)

    df_bt = BedTool.from_dataframe(df_).sort()
    df_ = df_bt.merge(d=0).to_dataframe()
    df_["gene_symbol"] = gene_symbol[0]
    df_["score"] = "."
    df_["strand"] = strand[0]
    df_ = df_.rename(columns={"chrom": "seqname", "start": "promTSS_left", "end":"promTSS_right"})
    
    return df_



def load_and_process_ensembl_gff3_file(file, n_downstream=100, n_upstream=1000, transcript_filtering="auto", clip_negative=True):
    # Load gff file. Comments rows are skipped.
    lines = []
    with open(file, "r") as f:
        for i, l in enumerate(f.readlines()):
            if l.startswith("#"):
                pass
            else:
                lines.append(l.replace("\n", "").split("\t"))
    df = pd.DataFrame(lines)


    # Data format adjustment 1
    df.columns = ["seqname", "source", "feature", "start", "end", "score",
                  "strand", "frame", "attribute"]

    df["start"] = df["start"].astype("int")
    df["end"] = df["end"].astype("int")

    # Data format adjustment 2
    ## The attribute column includes detailed information. Let's extract information and store them as new columns. 
    annot = pd.DataFrame([parse_ens(i) for i in tqdm(df["attribute"])])
    df = pd.concat([df, annot], axis=1)
    
    # Data format adjustment 2
    df["Parent_feature"] = [i.split(":")[0] for i in df.Parent.fillna("na:na")]
    df["Parent_id"] = [i.split(":")[1] for i in df.Parent.fillna("na:na")]


    # Data format adjustment 3
    df["gene_id_unified"] = df["gene_id"].values
    rows_non_gene = df["gene_id"].isna()
    df.loc[df.index[rows_non_gene], "gene_id_unified"] = df[rows_non_gene]["Parent_id"].values
    
    #return df
    # Split data into gene entry and transcript entry.
    df_gene = df[~rows_non_gene]
    df_gene = df_gene[["Name", "gene_id"]].rename(columns={"Name": "gene_symbol", "gene_id": "gene_id_unified"})
    df_transcript = df[df.Parent_feature == "gene"]
    df_transcript = pd.merge(df_transcript, df_gene, on="gene_id_unified", how="left")
            
    # We only use basic transcript, major mRNA isoform.
    
    if transcript_filtering == "auto":
        if "tag" in df.columns: # We only use basic transcript, major mRNA isoform.
            df_transcript = df_transcript[df_transcript.tag == "basic"]
            print("Found transcriptome tag information. Only 'basic' transcripts are used.")
        else:
            print("No transcriptome tag information found. Transfript filtering step is skipped. All transcripts are used.")
    
    elif transcript_filtering == "basic":
        if "tag" in df.columns: # We only use basic transcript, major mRNA isoform.
            df_transcript = df_transcript[df_transcript.tag == "basic"]
            print("Found transcriptome tag information. Only 'basic' transcripts are used.")
        else:
            ValueError("Could not perform basic transcript filtering because the gff3 file not contain 'tag' information. \n\
                  Please set transcript_filtering='auto'.\n\
                  With this mode, transcript filtering is not performed and all transcripts are used.")
    else:
        ValueError(f"transcript_filtering, {transcript_filtering} is not implemented")
    

    # Remove transcripts that are not annotated gene name.
    df_transcript = df_transcript[~df_transcript.gene_symbol.isna()]


    # Add PromoterTSS location. 
    df_transcript = get_tss_and_promoter_candidate_locus(df_transcript, 
                                         n_downstream=n_downstream, n_upstream=n_upstream, clip_negative=clip_negative)

    # Wrap up necessary information.
    result = df_transcript[["seqname", "promTSS_left", "promTSS_right",
                        "gene_symbol", "score", "strand"]]
    
    """# Merge overlapping peaks
    li = []
    for i in tqdm(result.gene_symbol.unique()):
        df_ = result[result.gene_symbol == i]
        if len(df_) == 1:
            li.append(df_)
        else:
            li.append(merge_overlapping_peaks(df_))
    result_merged = pd.concat(li, axis=0)"""

    return result

# 2. Install reference genome first.

We use genomepy to get genomic DNA sequence.
The first step is to install reference genome data.

We will use the genomepy function.
`genomepy.install_genome()`

We need (1) referenoce genome name and (2) provider.

Please see genomepy's documentation for more information. https://pypi.org/project/genomepy/


In [4]:
# Search for reference genome name and provider
!genomepy search "Cavia porcellus"

name         provider    accession          species            tax_id    other_info                                                  
Cavpor3.0    Ensembl     GCA_000151735.1    Cavia porcellus    10141     2016-12-ensembl/2017-07                                     
cavPor3      UCSC        na                 Cavia porcellus    10141     Feb. 2008 (Broad/cavPor3)                                   
Cavpor3.0    NCBI        GCA_000151735.1    Cavia porcellus    10141     The Genome Sequencing Platform, The Genome Assembly Team    
 ^
 Use name for genomepy install


In [7]:
# Install reference genome. You can skip this step if you already installed reference genome.
ref_genome = "Cavpor3.0"
provider = "Ensembl"
genomepy.install_genome(ref_genome, provider)

Using version 108
Genome download successful, starting post processing...

name: Cavpor3.0
local name: Cavpor3.0
fasta: /home/k/.local/share/genomes/Cavpor3.0/Cavpor3.0.fa


In [8]:
# Check referenoce genome installation status
genome_installation = ma.is_genome_installed(ref_genome=ref_genome)
genome_installation

True

# 3. Download genome annotation file; gff3 file, from Ensemble server. 
https://useast.ensembl.org/Cavia_porcellus/Info/Index?db=core
    

In [9]:
!wget http://ftp.ensembl.org/pub/release-105/gff3/cavia_porcellus/Cavia_porcellus.Cavpor3.0.105.gff3.gz

--2022-12-27 17:04:32--  http://ftp.ensembl.org/pub/release-105/gff3/cavia_porcellus/Cavia_porcellus.Cavpor3.0.105.gff3.gz
Resolving ftp.ensembl.org (ftp.ensembl.org)... 193.62.193.139
Connecting to ftp.ensembl.org (ftp.ensembl.org)|193.62.193.139|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12447679 (12M) [application/x-gzip]
Saving to: ‘Cavia_porcellus.Cavpor3.0.105.gff3.gz’

Cavia_porcellus.Cav 100%[===================>]  11.87M   620KB/s    in 20s     

2022-12-27 17:04:52 (617 KB/s) - ‘Cavia_porcellus.Cavpor3.0.105.gff3.gz’ saved [12447679/12447679]



In [10]:
!gunzip Cavia_porcellus.Cavpor3.0.105.gff3.gz

# 4. Process data to get TSS file.

In [15]:
# Load and process gff3 file.

file = "Cavia_porcellus.Cavpor3.0.105.gff3"
result = load_and_process_ensembl_gff3_file(file, n_downstream=100, 
                                            n_upstream=1000, 
                                            transcript_filtering="auto")


No transcriptome tag information found. Transfript filtering step is skipped. All transcripts are used.


In [15]:
# Check result
result

,seqname,promTSS_left,promTSS_right,gene_symbol,score,strand
4,DS562855.1,546865,547965,TMEM74,.,+
6,DS562855.1,867637,868737,EMC2,.,-
7,DS562855.1,849313,850413,EMC2,.,-
8,DS562855.1,933093,934193,5S_rRNA,.,+
10,DS562855.1,1132436,1133536,RSPO2,.,+
...,...,...,...,...,...,...
34835,MT,8879,9979,ND4L,.,+
34836,MT,9169,10269,ND4,.,+
34840,MT,10748,11848,ND5,.,+
34841,MT,13986,15086,ND6,.,-


In [16]:
# Save as bed file
result.to_csv(f"{ref_genome}_tss_info.bed", sep='\t', header=False, index=False)

# Test
Try to load DNA sequence using genomepy

In [17]:
# Load file
tss_file = BedTool(f"{ref_genome}_tss_info.bed").to_dataframe()
tss_file.head()

,chrom,start,end,name,score,strand
0,DS562855.1,546865,547965,TMEM74,.,+
1,DS562855.1,867637,868737,EMC2,.,-
2,DS562855.1,849313,850413,EMC2,.,-
3,DS562855.1,933093,934193,5S_rRNA,.,+
4,DS562855.1,1132436,1133536,RSPO2,.,+


In [18]:
tss_file.shape

(24875, 6)

In [19]:
# Get DNA sequence

peak_ids = tss_file["chrom"] + "_" + tss_file["start"].astype("str") + "_" + tss_file["end"].astype("str")
peak_ids = peak_ids.to_list()

fa = ma.peak2fasta(peak_ids, ref_genome=ref_genome)
fa

24875 sequences

In [20]:
# Show 3 sequences
n = 3

for i, (k, v) in enumerate(fa.items()):
    print(k, "\n", v, "\n")
    
    if i >= n - 1:
        break

DS562855.1_546866_547965 
 TTTGACTGGCTAAAAGAAAAACATGGGAAGAGTCAGTTGTGgtaatcccagcactcacaagtttgaggcaggaggattacctcaacttcgaatccagcctgggatacacagtgcattcaaggccaggctgaactacatagtgagaccctatctcaaaaaacctgggagagaaagagagagggcaagacTAAAAAAGAGGAGAAGGAAACAGGAAAAAAGACTTTGAAGGGAACAGGTTGCAAAGTCCCCGGAACCATGAGCTTATGACCTCTGTCCAACATAAAGGAAAATCAATGCAATAGTTAGTATAGGAAAGACAAACAACGAAAACAAGAAACGGGAGCATTTATTGAAGCTACTCACAAGCAGCTGAgtaccagcaatcgaactcatgccctagtgcttgtcaggcaggcgcttatgccgctgattaaatccccagcccCCTCATACTGTTTTGACTCATTATCATTCCTGCATCCCACAGATATTGCACACTGCACTAAGCAGAGTAACTTGGTGGAAGGACAACTTCCTCCTCTGAGGTCACCTTCCCTCATCTATGGGGAAAGCAATGGCTTTGGAAGCAAATTCCTCGATGCCATTTTAGAAGTGGCCGTTCATTGACActgtacctttgcttcctcctctgaaaaatgaggattaataaaatcaatctattttgatgacttcttaactatgaagacaaaacatgcaaattacctggcaagttgttgtaactaaatacacactcCTTCCAGCCAGTCCCTCCCTCATTCTGTGACCACCCTTGAAGTTGCAGGACCCCAAGGCAAGAGAGAAGTCATGGCTTTCCCCAGTGTAGGTAGGATCTACAGTGCAAGAGGACGTAATCCTGCCCCCAGGAGCCAGGCGGGATGCTCGGCGCCCTAGACTACAACTCCCAGGATGCTCGGCGGGAGGGGCCTAAccccgccctcaggccccgcccccc

Looks good

In [18]:
ls

1_make_tss_referenece_from_ensemble_gff3_file.ipynb
Cavpor3.0_tss_info.bed
peaks_example.csv
tss_annotation_using_custom_tss_data.ipynb
